In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

# To prevent CUBLAS_STATUS_ALLOC_FAILED problem in tensorflow 2, the follwing codes are necessary.

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [2]:
# Step 1: Data preparation

(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_labels = len(np.unique(y_train))

# One hot encoding

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshaping
image_size = x_train.shape[1]
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])

# normalizing

x_train = x_train/ 255.
x_test = x_test/ 255.

In [3]:
# Step 2: Model Construction 

# Hyperparameter setup

input_shape = [image_size,image_size,1]
batch_size=128
kernel_size=3
pool_size=2
dropout=0.3
epochs=20
filters = 64

# Model Construction using functional API

inputs = Input(shape=input_shape)
y= Conv2D(filters=filters, kernel_size=kernel_size, activation='relu')(inputs)
y= MaxPooling2D(pool_size=pool_size)(y) # By default, MaxPooling2D uses pool_size=2 , so the argument could be removed.
y= Conv2D(filters=filters, kernel_size=kernel_size, activation='relu')(y)
y= MaxPooling2D()(y) 

y= Conv2D(filters=filters, kernel_size=kernel_size, activation='relu')(y)

# image to vector before connecting to dense layer

y=Flatten()(y)

# dropout regularization

y=Dropout(dropout)(y)

# Connect to Final output layer

outputs = Dense(num_labels, activation='softmax')(y)

# Build the model by supplying inputs/outputs

model = Model(inputs=inputs, outputs = outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [4]:
# Step 3: Model Compile

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
# Step 4: Model Fit

import time
start_time=time.time()

model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=epochs, batch_size=batch_size)
print('Train Time in seconds: {:.2f}'.format(time.time()-start_time))

Epoch 1/20
469/469 [==============================] - 5s 6ms/step - loss: 0.6397 - accuracy: 0.8022 - val_loss: 0.0557 - val_accuracy: 0.9828
Epoch 2/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0803 - accuracy: 0.9753 - val_loss: 0.0364 - val_accuracy: 0.9879
Epoch 3/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0532 - accuracy: 0.9835 - val_loss: 0.0316 - val_accuracy: 0.9897
Epoch 4/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0453 - accuracy: 0.9863 - val_loss: 0.0293 - val_accuracy: 0.9897
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0367 - accuracy: 0.9881 - val_loss: 0.0239 - val_accuracy: 0.9914
Epoch 6/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0312 - accuracy: 0.9895 - val_loss: 0.0238 - val_accuracy: 0.9921
Epoch 7/20
469/469 [==============================] - 2s 4ms/step - loss: 0.0260 - accuracy: 0.9922 - val_loss: 0.0245 - val_accuracy: 0.9915
Epoch 

In [6]:
# Step 5: Model Evaluation

__,acc= model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test Accuracy: {:.2f}%'.format(100*acc))

79/79 [==============================] - 0s 2ms/step - loss: 0.0261 - accuracy: 0.9924
Test Accuracy: 99.24%
